# Music Classification using Neural Networks
by: Team 22
***
This notebook will serve as the 'main' training & classification point of network. It will be kept as easy-to-read as possible by importing necessary pre-processing, data loaders, and other helper functions that do not need to be tuned with the network.